In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/next-concept-predictor/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [39]:
# hyperparameters
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 500 
eval_interval = 50 
learning_rate = 3e-4
eval_iters = 20
n_embd = 64
n_head = 4
n_layer = 2
dropout = 0.1
gamma_0 = 0.5 ########################################################################################################################
t_max = 16 ########################################################################################################################
delta_gamma = (1 - gamma_0)/t_max
# ------------
print("maximum possible sequence length including concept vecs: ",block_size + t_max)
print("delta_gamma: ", delta_gamma)

maximum possible sequence length including concept vecs:  48
delta_gamma:  0.03125


In [5]:
with open('ng-video-lecture/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars, vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [7]:
chars_ = chars.copy()
chars_.append('') # our empty token
print(chars_, len(chars)) 
print(f"the new character is: '{chars_[65]}'")# index it with 65

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ''] 65
the new character is: ''


In [8]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [9]:
print(stoi, itos)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64} {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [10]:
# create a mapping from integers to characters including our empty character
itos_ = { i:ch for i,ch in enumerate(chars_) }
decode_ = lambda l: ''.join([itos_[i] for i in l]) # decoder: take a list of integers, output a string

In [11]:
print(itos_)
print(decode_([2, 12, 4, 65, 4]))

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z', 65: ''}
!?&&


In [12]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [13]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [14]:
# sample a batch of data
torch.manual_seed(420)
xb, yb = get_batch('train')
print(xb)
print(yb)
print(xb[:,0:8])
print(yb[:,8-1]) # now i've gotta use the i-1 as the index to get the correct target
print(yb[torch.arange(yb.size(0)), [0,1,2,3]]) # replace [0,1,2,3] for dynamic selection

tensor([[53, 59, 57,  1, 57, 58, 56, 43, 39, 49, 57,  0, 16, 53,  1, 50, 39, 41,
         43,  1, 58, 46, 43,  1, 57, 43, 60, 43, 56, 47, 52, 45],
        [52, 10,  0, 32, 46, 43, 56, 43, 44, 53, 56, 43,  1, 50, 43, 58,  1, 46,
         47, 51,  1, 40, 43,  1, 41, 53, 52, 57, 59, 50, 10,  1],
        [59, 52, 42, 43, 56, 40, 43, 39, 56, 47, 52, 45,  1, 53, 44,  1, 46, 47,
         57,  1, 44, 53, 56, 58, 59, 52, 43,  6,  0, 13, 57,  1],
        [42, 53,  1, 39, 52, 42,  1, 42, 53,  1, 47, 58,  1, 49, 47, 52, 42, 50,
         63,  6,  1, 45, 43, 52, 58, 50, 43,  1, 57, 47, 56, 57]],
       device='mps:0')
tensor([[59, 57,  1, 57, 58, 56, 43, 39, 49, 57,  0, 16, 53,  1, 50, 39, 41, 43,
          1, 58, 46, 43,  1, 57, 43, 60, 43, 56, 47, 52, 45,  1],
        [10,  0, 32, 46, 43, 56, 43, 44, 53, 56, 43,  1, 50, 43, 58,  1, 46, 47,
         51,  1, 40, 43,  1, 41, 53, 52, 57, 59, 50, 10,  1, 58],
        [52, 42, 43, 56, 40, 43, 39, 56, 47, 52, 45,  1, 53, 44,  1, 46, 47, 57,
          1, 

In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, last_hidden_state = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out
# i don't think i need to edit this one bc the loss will be done inside model()

In [16]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [17]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        self.sa = nn.MultiheadAttention(embed_dim=n_embd, num_heads=n_head)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self._sa_forward(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

    def _sa_forward(self, x):
        # MultiheadAttention expects inputs in (T, B, D) format.
        x = x.transpose(0, 1)  # Convert (B, T, D) to (T, B, D)
        attn_output, _ = self.sa(x, x, x)
        return attn_output.transpose(0, 1)  # Convert back to (B, T, D)

In [70]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, logits, target, mask=None):
        print("logits: ", logits.shape)
        print("target: ", target.shape)
        
        # If no mask is provided, create a default mask of ones
        # this scenario reduced to regular F.cross_entropy(logits, targets)
        if mask is None:
            mask = torch.ones_like(target, dtype=torch.float32)
            
        # Apply LogSoftmax
        log_probs = F.log_softmax(logits, dim=1)
        
        # Gather the log probabilities corresponding to the true classes
        gathered_probs = log_probs.gather(1, target.unsqueeze(1)).squeeze(1)

        # Apply the mask
        masked_probs = gathered_probs * mask

        # Compute the average loss over unmasked entries
        # Use A.sum() to count the number of unmasked entries
        loss = -torch.sum(masked_probs) / torch.clamp(mask.sum(), min=1)

        return loss

# Usage
#test = CustomLoss()
#loss = test(logits, target, mask)
# or, doing this reduces to F.cross_entropy()
#loss = test(logits, target)

In [88]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # learnable positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.apply(self._init_weights)
        
        # Initialize the custom loss function
        self.custom_loss = CustomLoss()

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,D)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,D)
        x = tok_emb + pos_emb # (B,T,D)
        x = self.blocks(x) # (B,T,D)
        x = self.ln_f(x) # (B,T,D)
        
        # Use the transposed embedding matrix for the output layer
        logits = x @ self.token_embedding_table.weight.t() # (B,T,D)@(D,V)=(B,T,V)
        
        if targets is None:
            loss = None
        else:
            # Use the custom loss function. u can add mask later once we have it
            loss = self.custom_loss(logits[:,-1,:].squeeze(dim=1), targets[:,-1])#, mask)
            # ok so problem here, our custom loss function is only looking at last part 
            # in target but regular F.cross_entropy() i think can compare everything in 

        # Splice out the last embedding vector from x
        last_hidden_state = x[:, -1, :] # (B, D)

        return logits, loss, last_hidden_state

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss, last_hidden_state = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, D)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, D)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [89]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')

106.304 K parameters


In [90]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [91]:
for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    ### conditionals ###
    if iter == 0:
        # initializing conditionals
        Gamma_i = torch.ones((batch_size))*gamma
        C_i = torch.zeros((batch_size, iter+1, n_embd))
        not_A_i = torch.ones((batch_size,iter+1), dtype=torch.long)
    #else: # bring this back once i have stuff to put in here
        # grab the conditionals calculated last run
        #Gamma_i, C_i, not_A_i = Gamma_iplus1, C_iplus1, not_A_iplus1
    
    # get dynamic indices from not_A_i:
    # use dynamic indices to select training indices
    #s_iplus1 = yb[torch.arange(yb.size(0)), not_A_i.sum(dim=1)]
    # wait i think this needs to go inside generate()
    
    # train
    logits, loss, last_hidden_state = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    ######################## the generate() does all at once instead of iteratively
    ###################### and i need to solve that before i can do conditionals
    ### conditionals ###
    #P = F.softmax(logits, dim=-1)
    #P_max = torch.max(P, dim=1).values
    
    #A_i_bar = torch.zeros(P_max.shape[0])
    #A_i_bar[P_max <= gamma_0] = 1
    
    #not_A_i_bar = torch.zeros(P_max.shape[0])
    #not_A_i_bar[P_max > gamma_0] = 1
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    

logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])

logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])

logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])

step 250: train loss 3.2862, val loss 3.5671
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target: 

logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])

logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target:  torch.Size([4])
step 450: train loss 3.0840, val loss 3.1534
logits:  torch.Size([4, 65])
target:  torch.Size([4])
logits:  torch.Size([4, 65])
target: 

In [50]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=64)[0].tolist()))


;T.iaKX,waJcr:,&! wZB!I
GWWgpv:VavWXFwa:l3hgcsFnsda&TWciKxvxpqCb


In [35]:
open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=256)[0].tolist()))

257